In [1]:
using Random

In [2]:
function euclidean_norm(vec::Vector)
    return sqrt(sum(vec.^2))
end

function generate_vector(l::Int, r::Int, n::Int)
    return rand(n) .* (r - l) .+ l
end

function generate_rhs_vector(A, B, C, x)
    n = length(x)
    d = zeros(n)
    d[1] = B[1] * x[1] + C[1] * x[2]
    for i in 2:n - 1
        d[i] = A[i - 1] * x[i - 1] + B[i] * x[i] + C[i] * x[i + 1]
    end
    d[n] = A[n - 1] * x[n - 1] + B[n] * x[n]
    
    return d
end

generate_rhs_vector (generic function with 1 method)

In [3]:
function forward(A, B, C, d)
    n = length(d)
    alphas = zeros(n)
    betas = zeros(n)
    alphas[1] = -C[1] / B[1]
    betas[1] = d[1] / B[1]
    for i in 2:n
        if i == n
            alphas[i] = 0.0
        else
            alphas[i] = -C[i] / (alphas[i - 1] * A[i - 1] + B[i])
        end
        betas[i] = (d[i] - A[i - 1] * betas[i - 1]) / (alphas[i - 1] * A[i - 1] + B[i])
    end
    
    return alphas, betas
end


function backward(alphas, betas)
    n = length(betas)
    x = zeros(n)
    x[n] = betas[n]
    for i in n-1:-1:1
        x[i] = alphas[i] * x[i + 1] + betas[i]
    end
    
    return x
end

function progonka(a, b, c, d)
    alphas, betas = forward(a,b,c,d)
    return backward(alphas, betas)
end

progonka (generic function with 1 method)

In [4]:
n = 10

a = generate_vector(-10,10,n-1)
b = generate_vector(-10,10,n)
c = generate_vector(-10,10,n-1)
x = ones(n)
d = generate_rhs_vector(a,b,c,x)

res = progonka(a,b,c,d)

println("Результат метода прогонки: $res")

shift = euclidean_norm(x - res)

println("Погрешность: $shift")

Результат метода прогонки: [1.0000000000000007, 0.9999999999999994, 1.0000000000000009, 1.0, 1.0, 1.0, 0.9999999999999998, 1.0, 1.0, 1.0]
Погрешность: 1.2609709600486848e-15
